In [ ]:
####将特征文件训练集测试集切割,分割是手动的，该模块仅用于获取各个label的标签index,因为已经知道顺序是1302
import numpy as np
import pandas as pd
params = {}
###特征提取方法
params['feature_path']=r'E:\文件夹\大三下\移动互联网课设\大作业\CWRU0319\feature\time_fre'
params['data_path'] = params['feature_path']+'.csv'
test = np.array(pd.read_csv(params['data_path'],keep_default_na=False))##后面这个参数可以解决空值带来的问题
test_x = test[:, :-1]
test_y=test[:,-1]#获取文件标签
every_file_number=np.zeros(4)#计数每一个测试文件的特征数量数组
#获得每个测试文件的特征数量
for file_label in test_y:
    every_file_number[int(file_label)]+=1
###按7：3切割，算一下，因为有小数等原因，肯定会有1，2条数据的差错，但没什么大影响
a=np.zeros(8)
a[0]=every_file_number[1]*0.7
a[1]=every_file_number[1]
a[2]=every_file_number[1]*0.7+every_file_number[3]*0.7
a[3]=every_file_number[1]*0.7+every_file_number[3]
a[4]=every_file_number[1]*0.7+every_file_number[3]*0.7+every_file_number[0]*0.7
a[5]=every_file_number[1]*0.7+every_file_number[3]*0.7+every_file_number[0]
a[6]=every_file_number[1]*0.7+every_file_number[3]*0.7+every_file_number[0]*0.7+every_file_number[2]*0.7
a[7]=every_file_number[1]*0.7+every_file_number[3]*0.7+every_file_number[0]*0.7+every_file_number[2]
#最终训练集标签值
print(a)
#测试集标签值
b=np.zeros(8)
b[0]=0
b[1]=every_file_number[1]*0.7
b[2]=every_file_number[1]*0.3
b[3]=every_file_number[1]*0.3+every_file_number[3]*0.7
b[4]=every_file_number[1]*0.3+every_file_number[3]*0.3
b[5]=every_file_number[1]*0.3+every_file_number[3]*0.3+every_file_number[0]*0.7
b[6]=every_file_number[1]*0.3+every_file_number[3]*0.3+every_file_number[0]*0.3
b[7]=every_file_number[1]*0.3+every_file_number[3]*0.3+every_file_number[0]*0.3+every_file_number[2]*0.7
#最终训练集标签值
print(b)

In [ ]:
###获取训练集测试集数据
params['train_data_path'] = params['feature_path']+'_train.csv'
train = np.array(pd.read_csv(params['train_data_path'],keep_default_na=False))##后面这个参数可以解决空值带来的问题
train_x = train[:, :-1]
train_y= train[:,-1]#获取文件标签
params['test_data_path'] =params['feature_path']+'_test.csv'
test = np.array(pd.read_csv(params['test_data_path'],keep_default_na=False))##后面这个参数可以解决空值带来的问题
test_x = test[:, :-1]
test_y=test[:,-1]#获取文件标签

In [ ]:
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV

In [ ]:
#RF模型优化
###通过不断迭代优化模型参数
###如果有的参数处于边界值，最好更改边界值在来一次，虽然即使没在边界值，也可能最优解在取值范围外
from sklearn.ensemble import RandomForestClassifier
# #有batime经过三次调参0.97
# params['n_estimators'] = 100
# params['max_depth'] = 11
# params['max_features'] = 3
# params['min_samples_split'] = 70
# params['min_samples_leaf'] = 10
#无batime
params['n_estimators']=70
params['max_depth']=13
params['min_samples_split']=70
params['min_samples_leaf']=1
params['max_features']=3
q1=params['n_estimators']
q2=params['max_depth']
q3=params['min_samples_split']
q4=params['min_samples_leaf']
q5=params['max_features']
def RF_n_estimators(params,train_x, train_y):
    param_test = {'n_estimators': range(10, 101, 10)}
    gsearch = GridSearchCV(
        estimator=RandomForestClassifier(
                    min_samples_split=params['min_samples_split'],
                    min_samples_leaf=params['min_samples_leaf'],
                    max_depth=params['max_depth'],
                    max_features=params['max_features'],
                    random_state=10,
                    ),
        param_grid=param_test,
        scoring='accuracy',
        cv=5)
    gsearch.fit(train_x, train_y)
    return gsearch.best_params_.get('n_estimators')

def RF_max_depth_min_samples_split(params,train_x, train_y):
    param_test = {'max_depth': range(3, 24, 2), 'min_samples_split': range(50, 201, 20)}
    gsearch = GridSearchCV(
        estimator=RandomForestClassifier(
            n_estimators=params['n_estimators'],
            max_features=params['max_features'],
            min_samples_leaf=params['min_samples_leaf'],
            oob_score=True,
            random_state=10),
        param_grid=param_test,
        scoring='accuracy',
        iid=False,
        cv=5)
    gsearch.fit(train_x, train_y)
    return gsearch.best_params_.get('max_depth'),gsearch.best_params_.get('min_samples_split')

def RF_min_samples_split_min_samples_leaf(params,train_x, train_y):
    param_test = {'min_samples_split': range(params['min_samples_split'], 200, 20), 'min_samples_leaf': range(1, 10, 1)}
    gsearch = GridSearchCV(
        estimator=RandomForestClassifier(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            max_features=params['max_features'],
            oob_score=True,
            random_state=10),
        param_grid=param_test,
        scoring='accuracy',
        iid=False,
        cv=5)
    gsearch.fit(train_x, train_y)
    return gsearch.best_params_.get('min_samples_split'),gsearch.best_params_.get('min_samples_leaf')

def RF_max_features(params,train_x, train_y):
    param_test = {'max_features': range(1, 10, 1)}
    gsearch = GridSearchCV(
        estimator=RandomForestClassifier(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            min_samples_split=params['min_samples_split'],
            min_samples_leaf=params['min_samples_leaf'],
                    random_state=10,
                    ),
        param_grid=param_test,
        scoring='accuracy',
        cv=5)
    gsearch.fit(train_x, train_y)
    return gsearch.best_params_.get('max_features')
#迭代次数
i=np.zeros(5)
while 1<2:
    ###归0
    q11=0
    q22=0
    q33=0
    q44=0
    q55=0
    params['n_estimators'] = RF_n_estimators(params,train_x, train_y)
    if params['n_estimators']!=q1:
        i[0]+=1
        print("n_estimators迭代次数:"+str(i[0]))
        q1=params['n_estimators']
        q11=1
    params['max_depth'],params['min_samples_split'] = RF_max_depth_min_samples_split(params,train_x, train_y)
    if params['max_depth']!=q2:
        i[1]+=1
        print("max_depth迭代次数:"+str(i[1]))
        q2=params['max_depth']
        q22=1    
    params['min_samples_split'],params['min_samples_leaf'] = RF_min_samples_split_min_samples_leaf(params,train_x, train_y)
    if params['min_samples_split']!=q3:
        i[2]+=1
        print("'min_samples_split:"+str(i[2]))
        q3=params['min_samples_split']
        q33=1  
    if params['min_samples_leaf']!=q4:
        i[3]+=1
        print("min_samples_leaf迭代次数:"+str(i[3]))
        q4=params['min_samples_leaf']
        q44=1  
    params['max_features'] = RF_max_features(params,train_x, train_y)
    if params['max_features']!=q5:
        i[4]+=1
        print("max_features迭代次数:"+str(i[4]))
        q5=params['max_features']
        q55=1  
    if q11==0 and q22==0 and q33==0 and q44==0 and q55==0:
        ###即所有参数已经不再变化,则不在优化
        break
print("params['n_estimators']="+str(params['n_estimators']))
print("params['max_depth']="+str(params['max_depth']))
print("params['min_samples_split']="+str(params['min_samples_split']))
print("params['min_samples_leaf']="+str(params['min_samples_leaf']))
print("params['max_features']="+str(params['max_features']))

##模型评估
rf = RandomForestClassifier(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        max_features=params['max_features'],
        min_samples_split=params['min_samples_split'],
        min_samples_leaf=params['min_samples_leaf'],
        oob_score=True,
        random_state=10
        )
rf.fit(train_x, train_y)
y_pred = rf.predict(test_x)
# 模型评价
f1 = f1_score(test_y, y_pred, average='macro')
acc = accuracy_score(test_y, y_pred)
print('f1 : ', f1)
print('accuracy : ', acc)

# ##result0.7:0.3
# params['n_estimators']=20
# params['max_depth']=11
# params['min_samples_split']=210
# params['min_samples_leaf']=3
# params['max_features']=3
# # f1 :  0.6928239891855987
# # accuracy :  0.9608948087431693
# ###result1.0来得到参数，在0.7：0.3来训练测试
# params['n_estimators']=70
# params['max_depth']=13
# params['min_samples_split']=70
# params['min_samples_leaf']=1
# params['max_features']=3
# # f1 :  0.8010062215899816
# # accuracy :  0.9748975409836066